In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#%matplotlib inline

from IPython.core.pylabtools import figsize
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import Counter

figsize(10, 8)
sns.set()

In [4]:
files = ['fileViews',
             'premium_cancellations',
             'premium_payments',
             'questions',
             'sessions',
             'students',
             'subjects',
             'cancellations_agg',
             'fileViews_agg',
             'payments_agg',
             'questions_agg',
             'sessions_agg',
             'subjects_agg',
             'usage_weekly',
         'usage_desktop_only',
         'usage_mobile_only'
             ]

datasets = {}
for file in files:
    datasets[file] = pd.read_csv(f"../data/processed/{file}.csv")  # ,parse_dates=[1,2],index_col=[0])
    print(f'*********{file}***********')
    print(datasets[file].shape)
    print(datasets[file].head(2))
    print("____________\n\n")

/home/ahgora/projetos/pd/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ahgora/projetos/pd/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*********fileViews***********
(3028988, 4)
  StudentId                                           FileName  \
0  12970655  Exercicios Resolvidos do Halliday sobre Rotaçã...   
1  12970655  Exercicios Resolvidos do Halliday sobre Rotaçã...   

                     ViewDate Studentclient  
0  2017-02-23 10:46:03.047000       Website  
1  2017-02-23 10:46:12.370000       Website  
____________


*********premium_cancellations***********
(844, 2)
   StudentId            CancellationDate
0   34129668  2016-05-05 19:08:31.937000
1   34129668  2016-05-05 22:48:20.830000
____________


*********premium_payments***********
(7276, 3)
   StudentId                 PaymentDate PlanType
0   12970655  2017-05-26 22:25:03.000000    Anual
1   12970655  2018-05-26 10:05:28.000000    Anual
____________


*********questions***********
(3906, 4)
   StudentId                QuestionDate  \
0   12970655  2013-09-04 14:09:25.963000   
1   12970655  2013-09-04 15:36:51.083000   

                               

In [150]:
fileViews = datasets.get('fileViews')
cancellation = datasets.get('premium_cancellations')
payment = datasets.get('premium_payments')
questions = datasets['questions']
sessions = datasets['sessions']
student = datasets.get('students')
subjects = datasets.get('subjects')
cancellation_agg = datasets.get('cancellations_agg')
fileViews_agg = datasets.get('fileViews_agg')
payments_agg = datasets.get('payments_agg')
questions_agg = datasets.get('questions_agg')
sessions_agg = datasets.get('sessions_agg')
subjects_agg = datasets.get('subjects_agg')
usage_weekly = datasets.get('usage_weekly')
usage_mobile = datasets.get('usage_mobile_only')
usage_desktop = datasets.get('usage_desktop_only')

In [126]:
max_time = sessions.SessionStartTime.max()
sessions_agg = sessions_agg[['StudentId','SessionStartTime']]
sessions_agg.columns = ['StudentId','session_count']

In [127]:
fileViews_agg = fileViews_agg[['StudentId','FileName']]
fileViews_agg.columns = ['StudentId','fileview_count']
fileViews_agg.drop([45975, 45976] ,inplace=True)
fileViews_agg.head()
fileViews_agg['StudentId'] = fileViews_agg.StudentId.astype(int)

/home/ahgora/projetos/pd/venv/lib/python3.7/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ahgora/projetos/pd/venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
questions_agg = questions_agg[['StudentId','QuestionDate']]
questions_agg.columns = ['StudentId','question_count']

In [ ]:
cancellation_agg = cancellation_agg[['StudentId','CancellationDate']]
cancellation_agg.columns = ['StudentId','cancelation_count']


In [156]:
subjects_agg = subjects_agg[['StudentId','SubjectName']]
subjects_agg.columns = ['StudentId','subject_count']

## Features
- StudentId
- registered_time(days)
- usage_weekly_count(com session)
- usage_weekly_mean(com session)
- count_sessions
- rate - sessions(qtd / tempobase)
- count_file_view
- Rate - fileview
- count_question
- rate - question
- Região - ohe - OK
- Estado - ohe - OK
- Device_type
- count_payment
- count_payment_mensal
- count_paymant_anual
- count_cancellation
- count_subject


In [129]:
def get_registered_time(df, max_time):
    tempos = []
    for student_reg_date in df.RegisteredDate:
        diff_time = pd.to_timedelta(pd.to_datetime(max_time) - pd.to_datetime(student_reg_date) ).days
        tempos.append(diff_time)
    return tempos

def get_region(df):

    norte = ['Acre','Amapa','Amazonas','Pará', 'Rondonia','Roraima','Tocantins']
    nordeste = ['Alagoas', 'Bahia','Ceara','Maranhão','Paraíba','Pernambuco', 'Piauí', 'Rio Grande do Norte', 'Sergipe' ]
    centro_oeste = ['Goias','Mato Grosso','Mato Grosso do Sul' ,'Distrito Federal']
    sul = ['Rio Grande do Sul','Santa Catarina','Paraná']
    sudeste = ['Espirito Santo','Minas Gerais','Rio de Janeiro','São Paulo']
    na = ['NA']

    regions = []
    for student_state in df.State.fillna("NA"):
        region = next(n for n,v in filter(lambda t: isinstance(t[1],list), locals().items()) if student_state in v)
        regions.append(region)
    return regions

In [130]:
student['registered_time'] = get_registered_time(student,max_time)
student_usage = pd.merge(student.head(50),
              usage_weekly,
              left_on='Id',
              right_on='StudentId',
              how='left')

In [131]:
student_usage_sessions = pd.merge(student_usage,
                                  sessions_agg,
                                  on='StudentId' ,
              how='left')

In [134]:
student_usage_sessions = pd.merge(student_usage_sessions,
                                  sessions_agg,
                                  on='StudentId' ,
                                  suffixes= '',
                                  how='left')
student_usage_sessions['session_rate'] = student_usage_sessions['session_count']/student_usage_sessions['registered_time']

/home/ahgora/projetos/pd/venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  """


In [135]:
student_usage_sessions_fileViews= pd.merge(student_usage_sessions,
                                  fileViews_agg[['StudentId','fileview_count']],
                                  on='StudentId' ,
              how='left')

student_usage_sessions_fileViews['fileview_rate'] = student_usage_sessions_fileViews['fileview_count']/student_usage_sessions_fileViews['registered_time']

In [136]:
student_usage_sessions_fileViews_question = pd.merge(student_usage_sessions_fileViews,
                                  questions_agg[['StudentId','question_count']],
                                  on='StudentId' ,
              how='left')

student_usage_sessions_fileViews_question['question_rate'] = student_usage_sessions_fileViews_question['question_count']/student_usage_sessions_fileViews_question['registered_time']

In [137]:
student_usage_sessions_fileViews_question['region'] = get_region(student_usage_sessions_fileViews_question)


In [138]:
list_desktop = usage_desktop.StudentId.unique()
list_mobile = usage_mobile.StudentId.unique()

student_usage_sessions_fileViews_question['mobile'] = 0
student_usage_sessions_fileViews_question['desktop'] = 0

student_usage_sessions_fileViews_question.loc[student_usage_sessions_fileViews_question.StudentId.isin(list_mobile),'mobile'] = 1
student_usage_sessions_fileViews_question.loc[student_usage_sessions_fileViews_question.StudentId.isin(list_desktop),'desktop'] = 1

In [139]:
df_payment_total = payments_agg.groupby(['StudentId']).PaymentDate.sum().reset_index()
df_payment_total_mensal = payments_agg.loc[payments_agg.PlanType=='Mensal'].groupby(['StudentId']).PaymentDate.sum().reset_index()
df_payment_total_anual = payments_agg.loc[payments_agg.PlanType=='Anual'].groupby(['StudentId']).PaymentDate.sum().reset_index()
df_payment_total.columns = ['StudentId',"payment_total"]
df_payment_total_mensal.columns = ['StudentId',"payment_monthly"]
df_payment_total_anual .columns= ['StudentId',"payment_yearly"]


In [143]:
student_usage_sessions_fileViews_question_payment = pd.merge(student_usage_sessions_fileViews_question,
                                  df_payment_total,
                                  on='StudentId',
              how='left')
student_usage_sessions_fileViews_question_payment_month = pd.merge(student_usage_sessions_fileViews_question_payment,
                                  df_payment_total_mensal,
                                  on='StudentId',
              how='left')
student_usage_sessions_fileViews_question_payment_year = pd.merge(student_usage_sessions_fileViews_question_payment_month,
                                  df_payment_total_anual,
                                  on='StudentId',
              how='left')

,Id,RegisteredDate,UniversityName,CourseName,State,SignupSource,City,StudentClient,registered_time,usage_weekly_count,...,fileview_count,fileview_rate,question_count,question_rate,region,mobile,desktop,payment_total,payment_monthly,payment_yearly
0,12970655,2012-05-29 15:22:27.490000,PUC-RIO,Administração,Rio de Janeiro,Email,0,0,2200,46.0,...,45.0,0.020455,6.0,0.002727,sudeste,1,1,20.0,18.0,2.0
1,59873654,2012-09-03 20:32:04.853000,UFF,Direito do Trabalho e Segurança Social,Rio de Janeiro,Facebook,Rio de Janeiro,0,2103,10.0,...,19.0,0.009035,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0
2,3664695,2012-09-10 20:26:30.253000,UNB,Enfermagem,Distrito Federal,Facebook,0,0,2096,4.0,...,7.0,0.003340,0.0,0.000000,centro_oeste,0,1,0.0,0.0,0.0
3,15207697,2012-09-05 15:31:08.090000,UERJ,Engenharia de Produção Mecânica,Rio de Janeiro,Facebook,Resende,0,2101,3.0,...,5.0,0.002380,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0
4,36988693,2012-10-15 21:34:51.143000,UFU,Engenharia Elétrica,0,Facebook,0,0,2061,21.0,...,46.0,0.022319,0.0,0.000000,na,0,1,0.0,0.0,0.0
5,1001195,2012-10-15 21:35:05.540000,UFES,Química,Espirito Santo,Facebook,0,0,2061,6.0,...,10.0,0.004852,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0
6,69894068,2012-08-27 14:19:58.723000,PUC-MINAS,Direito,Minas Gerais,Facebook,Contagem,0,2110,5.0,...,9.0,0.004265,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0
7,98291487,2012-09-26 02:29:32.973000,UFABC,Engenharia Biomédica,São Paulo,Facebook,São Bernardo do Campo,0,2081,2.0,...,2.0,0.000961,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0
8,9448057,2012-12-03 19:55:03.750000,UFSCAR,Biotecnologia,0,Facebook,0,0,2012,2.0,...,3.0,0.001491,0.0,0.000000,na,0,1,0.0,0.0,0.0
9,32843000,2012-08-27 15:18:35.750000,PUC-MINAS,Engenharia Civil,Minas Gerais,Facebook,Belo Horizonte,0,2110,5.0,...,0.0,0.000000,0.0,0.000000,sudeste,0,0,0.0,0.0,0.0


In [ ]:
student_usage_sessions_fileViews_question_payment_year_cancellation = pd.merge(student_usage_sessions_fileViews_question_payment_year,
                                  cancellation_agg,
                                  on='StudentId',
              how='left')

In [158]:
student_usage_sessions_fileViews_question_payment_year_cancellation_subject = pd.merge(student_usage_sessions_fileViews_question_payment_year_cancellation,
                                  subjects_agg,
                                  on='StudentId',
              how='left')

In [159]:
student_usage_sessions_fileViews_question_payment_year_cancellation_subject.fillna(0)

,Id,RegisteredDate,UniversityName,CourseName,State,SignupSource,City,StudentClient,registered_time,usage_weekly_count,...,question_count,question_rate,region,mobile,desktop,payment_total,payment_monthly,payment_yearly,cancelation_count,subject_count
0,12970655,2012-05-29 15:22:27.490000,PUC-RIO,Administração,Rio de Janeiro,Email,0,0,2200,46.0,...,6.0,0.002727,sudeste,1,1,20.0,18.0,2.0,0.0,6.0
1,59873654,2012-09-03 20:32:04.853000,UFF,Direito do Trabalho e Segurança Social,Rio de Janeiro,Facebook,Rio de Janeiro,0,2103,10.0,...,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0,0.0,5.0
2,3664695,2012-09-10 20:26:30.253000,UNB,Enfermagem,Distrito Federal,Facebook,0,0,2096,4.0,...,0.0,0.000000,centro_oeste,0,1,0.0,0.0,0.0,0.0,5.0
3,15207697,2012-09-05 15:31:08.090000,UERJ,Engenharia de Produção Mecânica,Rio de Janeiro,Facebook,Resende,0,2101,3.0,...,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0,0.0,5.0
4,36988693,2012-10-15 21:34:51.143000,UFU,Engenharia Elétrica,0,Facebook,0,0,2061,21.0,...,0.0,0.000000,na,0,1,0.0,0.0,0.0,0.0,5.0
5,1001195,2012-10-15 21:35:05.540000,UFES,Química,Espirito Santo,Facebook,0,0,2061,6.0,...,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0,0.0,4.0
6,69894068,2012-08-27 14:19:58.723000,PUC-MINAS,Direito,Minas Gerais,Facebook,Contagem,0,2110,5.0,...,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0,0.0,5.0
7,98291487,2012-09-26 02:29:32.973000,UFABC,Engenharia Biomédica,São Paulo,Facebook,São Bernardo do Campo,0,2081,2.0,...,0.0,0.000000,sudeste,0,1,0.0,0.0,0.0,0.0,0.0
8,9448057,2012-12-03 19:55:03.750000,UFSCAR,Biotecnologia,0,Facebook,0,0,2012,2.0,...,0.0,0.000000,na,0,1,0.0,0.0,0.0,0.0,5.0
9,32843000,2012-08-27 15:18:35.750000,PUC-MINAS,Engenharia Civil,Minas Gerais,Facebook,Belo Horizonte,0,2110,5.0,...,0.0,0.000000,sudeste,0,0,0.0,0.0,0.0,0.0,10.0
